In [131]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import pandas as pd
import numpy as np
import cv2
import os


In [132]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 

In [133]:
csv_file = 'english.csv'
data = pd.read_csv(csv_file)

In [134]:
from PIL import Image

def load_and_preprocess_image(Img, target_size=(128, 128)):
    with Image.open(Img) as img:
        img = img.resize(target_size)
        image = np.array(img) / 255.0
    return image

In [135]:
images = []
labels = []
for index, row in data.iterrows():
    image_path = row['image']
    label = row['label']
    image = load_and_preprocess_image(image_path)
    images.append(image)
    labels.append(label)  

In [136]:
images = np.array(images, dtype=np.float32)

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = np.array(labels, dtype=np.int32)

In [137]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [138]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

d:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [139]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test), batch_size=32)

loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 154ms/step - accuracy: 0.0198 - loss: 4.1674 - val_accuracy: 0.0411 - val_loss: 4.0078
Epoch 2/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.0769 - loss: 3.8556 - val_accuracy: 0.2419 - val_loss: 3.1827
Epoch 3/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.2109 - loss: 3.0293 - val_accuracy: 0.4164 - val_loss: 2.3533
Epoch 4/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 165ms/step - accuracy: 0.3759 - loss: 2.2521 - val_accuracy: 0.5235 - val_loss: 1.8559
Epoch 5/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 166ms/step - accuracy: 0.5099 - loss: 1.7854 - val_accuracy: 0.5953 - val_loss: 1.5768
Epoch 6/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 166ms/step - accuracy: 0.5628 - loss: 1.4964 - val_accuracy: 0.6276 - val_loss: 1.4008
Epoch 7/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 167ms/step - accuracy: 0.6383 - loss: 1.1964 - val_accuracy: 0.6437 - val_loss: 1.3592
Epoch 8/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 15s 169ms/step - accuracy: 0.6945 - loss: 1.0011 - val_accu

In [141]:
test_image_path = 'a.png'
test_image = load_and_preprocess_image(test_image_path)
test_image = np.expand_dims(test_image, axis=0)  

In [142]:
predictions = model.predict(test_image)
predicted_class = np.argmax(predictions, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step


In [145]:
decoded_class = label_encoder.inverse_transform(predicted_class)
print(f'Predicted class: {decoded_class[0]}')

Predicted class: k
